# Build Python Models to Predict Auto Loan Default

- XGBoost:  Serialize with Pickle 


In [29]:
#import csv
import pandas as pd
import numpy as np
import os, shutil # used to create necessary folders
import pickle
import sklearn
from xgboost import XGBClassifier 
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 

import warnings
warnings.filterwarnings("ignore")

In [30]:
auto = pd.read_csv('Data_orig/AutoLoan.csv')

auto.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)

auto

 

,Loan_ID,BAD,LOAN,LOANDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,772418,1,1100,25860.0000,39025.000000,CarImp,Other,10.500000,0.00000,0.000000,94.366667,1.000000,9.000000,33.779915
1,477724,1,1300,70053.0000,68400.000000,CarImp,Other,7.000000,0.00000,2.000000,121.833333,0.000000,14.000000,33.779915
2,150746,1,1500,13500.0000,16700.000000,CarImp,Other,4.000000,0.00000,0.000000,149.466667,1.000000,10.000000,33.779915
3,108584,1,1500,73760.8172,101776.048741,NaN,NaN,8.922268,0.25457,0.449442,179.766275,1.186055,21.296096,33.779915
4,321534,0,1700,97800.0000,112000.000000,CarImp,Office,3.000000,0.00000,0.000000,93.333333,0.000000,14.000000,33.779915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,346389,0,88900,57264.0000,90185.000000,DebtCon,Other,16.000000,0.00000,0.000000,221.808718,0.000000,16.000000,36.112347
5956,255372,0,89000,54576.0000,92937.000000,DebtCon,Other,16.000000,0.00000,0.000000,208.692070,0.000000,15.000000,35.859971
5957,668928,0,89200,54045.0000,92924.000000,DebtCon,Other,15.000000,0.00000,0.000000,212.279697,0.000000,15.000000,35.556590
5958,233047,0,89800,50370.0000,91861.000000,DebtCon,Other,14.000000,0.00000,0.000000,213.892709,0.000000,16.000000,34.340882


In [31]:
auto.dtypes

Loan_ID      int64
BAD          int64
LOAN         int64
LOANDUE    float64
VALUE      float64
REASON      object
JOB         object
YOJ        float64
DEROG      float64
DELINQ     float64
CLAGE      float64
NINQ       float64
CLNO       float64
DEBTINC    float64
dtype: object

In [32]:
target         = ["BAD"]
numeric_inputs = ['LOAN', 'LOANDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ','CLNO', 'DEBTINC']
class_inputs   = ['REASON', 'JOB']
 
auto           =auto.dropna()
auto.REASON.replace(np.NaN,'CarImp',regex = True, inplace=True)
auto.JOB.replace(np.nan,'Other',regex = True, inplace=True)

 
x              = auto.drop(['BAD', 'Loan_ID'], axis=1)
y              = auto[target].astype("category")


 
 

In [33]:
#Write out 5 test records
auto2 =auto.head()
auto2.drop(['BAD'], axis=1, inplace=True)
auto2.to_csv('Data_orig/autoloan2_ds2_test.csv', index=False)


# Split Dataset Into Training and Test Set

In [34]:
# Split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test

print("train size: " + str(x_train.shape))
print("test size: " + str(x_test.shape))
print("train sample")

 
 

train size: (3875, 12)
test size: (1661, 12)
train sample


# Custom preprocessing for categorical inputs to use in XGB model

In [35]:
 
ohe = OneHotEncoder(sparse = False, handle_unknown="ignore")
enc = ohe.fit_transform(x_train[class_inputs])
enc2 = ohe.transform(x_test[class_inputs])

 
x_train[ohe.get_feature_names(class_inputs)] = pd.DataFrame(enc, index=x_train.index)
x_train.drop(class_inputs,1,inplace=True)

x_test[ohe.get_feature_names(class_inputs)] = pd.DataFrame(enc2, index=x_test.index)
x_test.drop(class_inputs,1,inplace=True)


print(x_train.shape)
print(x_test.shape)


 
 


(3875, 18)
(1661, 18)


In [36]:
#Pickle the one hot encoding
pickle.dump(ohe, open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder_lorry.pickle', 'wb'))
 


# Train the XGBoost Model

In [37]:
booster = xgb.XGBClassifier(max_depth=3,
                        booster='dart',
                        eta=0.3,
                        silent=1,
                        n_estimators=100)


# Train XGBoost Classifer
booster.fit(x_train,y_train)

modelName = "AutoLoan_XGBoost_Lorry"
 

[12:39:12] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:39:12] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# Pickle the trained XGB Model  

In [38]:
#pickle.dump(model, open(project_dir+"/HMEQ_XGB.pickle", 'wb'))
pickle.dump(booster, open("/sasinside/userdata/gegrab/resources/hmeq"+"/"+ modelName + ".pickle", 'wb'))
 

# Write Out Parameters for Copying

In [39]:
# Define input variables and perform formatting to match with score code file

input_params = ''
for col in auto.columns:
    input_params += col
    if col != auto.columns[-1]:
        input_params += ', '

input_cols = ''
for col in auto.columns:
    input_cols += "\"" + col + "\""
    if col != auto.columns[-1]:
        input_cols += ', '


print(input_cols)
print("")
print(input_params)

"Loan_ID", "BAD", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"

Loan_ID, BAD, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC


# Test XGBoost Score Code

In [40]:
def scoreXGB(Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC):
    "Output: P_BAD"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ["Loan_ID", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    
    inputArray.REASON.replace(np.nan,'HomeImp',regex = True, inplace=True)
    inputArray.JOB.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['Loan_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder_lorry.pickle")
      
     
    sess = pickle.load(open("/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_XGBoost_Lorry.pickle", 'rb'))

    P_BAD = float(sess.predict_proba(test_data)[0,0])
    
    
     
    
     
     
    return (P_BAD)



In [41]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/autoloan2_ds2_test.csv') 
  
 
for i in range(5):
    print(scoreXGB(**X_test2.iloc[i]))

0.01895761489868164
0.20398783683776855
0.00891488790512085
0.5066103935241699
0.20350873470306396


# Write Out XGBoost Score Code

In [42]:
score_code = """
def scoreXGB(Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC):
    "Output: P_BAD"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ["Loan_ID", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    
    inputArray.REASON.replace(np.nan,'HomeImp',regex = True, inplace=True)
    inputArray.JOB.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['Loan_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder_lorry.pickle")
      
     
    sess = pickle.load(open("/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_XGBoost_Lorry.pickle", 'rb'))

    P_BAD = float(sess.predict_proba(test_data)[0,0])
    
    
     
    
     
     
    return (P_BAD)"""


f = open('Data_orig/AutoLoan_XGB_Lorry.py',"w+")
f.write(score_code)
f.close()